### Loading Data Cash in circulation###

**Import**

In [354]:
import requests
import pandas as pd
import json
import seaborn as sns

**Request API**

In [355]:
response = requests.get("https://olinda.bcb.gov.br/olinda/servico/mecir_dinheiro_em_circulacao/versao/v1/odata/informacoes_diarias")
print(response.status_code)

200


**Convert from json to Dataframe**

In [356]:
r=response.json()
r.keys()

dict_keys(['@odata.context', 'value'])

We interesting in the values inside value.

In [357]:
flow = pd.DataFrame(r['value'])

In [358]:
flow.head()

,Data,Quantidade,Valor,Denominacao,Especie
0,1994-10-03,692701959,6.927020e+06,0.01,Moedas
1,1994-10-03,462277579,2.311388e+07,0.05,Moedas
2,1994-10-03,404559065,4.045591e+07,0.10,Moedas
3,1994-10-03,1492870,3.732175e+05,0.25,Moedas
4,1994-10-03,278901842,1.394509e+08,0.50,Moedas


In [359]:
flow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80995 entries, 0 to 80994
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Data         80995 non-null  object 
 1   Quantidade   80995 non-null  int64  
 2   Valor        80995 non-null  float64
 3   Denominacao  80995 non-null  object 
 4   Especie      80995 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 3.1+ MB


**Fix data format**

Data is an object dtype, but for this analysis is more convenient a DateTime format.

In [360]:
flow['Data']=pd.to_datetime(flow['Data'])  


Denominacao Should be a numeric value,but it is an object what is a redflag for inconsistency even thought we want to keep as an object.

In [361]:
flow['Denominacao'].unique()

array(['0.01', '0.05', '0.10', '0.25', '0.50', '1.00', '5.00', '10.00',
       '50.00', '100.00', '2.00', '20.00', '200.00'], dtype=object)

In [362]:
flow['Data']

0       1994-10-03
1       1994-10-03
2       1994-10-03
3       1994-10-03
4       1994-10-03
           ...    
80990   2020-09-17
80991   2020-09-17
80992   2020-09-17
80993   2020-09-17
80994   2020-09-17
Name: Data, Length: 80995, dtype: datetime64[ns]

No problems with Denominacao.

In [363]:
flow.head()

,Data,Quantidade,Valor,Denominacao,Especie
0,1994-10-03,692701959,6.927020e+06,0.01,Moedas
1,1994-10-03,462277579,2.311388e+07,0.05,Moedas
2,1994-10-03,404559065,4.045591e+07,0.10,Moedas
3,1994-10-03,1492870,3.732175e+05,0.25,Moedas
4,1994-10-03,278901842,1.394509e+08,0.50,Moedas


In [364]:
flow=flow['Data'].dt.to_period('M') 

### Loading inflation ###

In [365]:
inflation=pd.read_csv('C:\\Users\\tiago\\OneDrive\\Documentos\\Nota 200 conto\\tabela1737.csv')

In [366]:
inflation.head()

,"Tabela 1737 - IPCA - Série histórica com número-índice, variação mensal e variações acumuladas em 3 meses, em 6 meses, no ano e em 12 meses (a partir de dezembro/1979)",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,Mês,Variável,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,IPCA - Número-índice (base: dezembro de 1993 =...,IPCA - Variação mensal (%),IPCA - Variação acumulada em 3 meses (%),IPCA - Variação acumulada em 6 meses (%),IPCA - Variação acumulada no ano (%),IPCA - Variação acumulada em 12 meses (%)
2,Brasil,dezembro 1979,7.6183E-09,...,...,...,...,...
3,NaN,janeiro 1980,8.1223E-09,6.62,...,...,6.62,...
4,NaN,fevereiro 1980,8.4973E-09,4.62,...,...,11.54,...


We are insterested in the columns Unnamed: 1 and Unnamed: 3, but the column labels are shifted let's adress this problem. 

In [367]:
inflation=inflation.iloc[2:,[1,3]]

In [368]:
inflation.columns=['year/month','monthly inflation change']

In [369]:
inflation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490 entries, 2 to 491
Data columns (total 2 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   year/month                489 non-null    object
 1   monthly inflation change  489 non-null    object
dtypes: object(2)
memory usage: 7.8+ KB


We are going to use year/month as the key for merging with cash circulation so we are going to change to the date format mm/yyyy

In [370]:
months={'janeiro':'01-','fevereiro':'02-','março':'03-','abril':'04-','maio':'05-','junho':'06-','julho':'07-',
      'agosto':'08-','setembro':'09-','outubro':'10-','novembro':'11-','dezembro':'12-'}      

In [371]:
for key in months.keys():
    inflation['year/month']=inflation['year/month'].str.replace(key,months[key])
inflation['year/month']=inflation['year/month'].str.replace(' ','')

In [372]:
inflation['year/month']=pd.to_datetime(inflation['year/month']).dt.to_period('M') 


In [373]:
inflation

,year/month,monthly inflation change
2,1979-12,...
3,1980-01,6.62
4,1980-02,4.62
5,1980-03,6.04
6,1980-04,5.29
...,...,...
487,2020-05,-0.38
488,2020-06,0.26
489,2020-07,0.36
490,2020-08,0.24


**Merge data**

In [374]:
merged=pd.merge(flow, inflation, how='inner', left_on='Data', right_on='year/month')

In [375]:
merged.drop(columns='year/month',inplace=True)

In [377]:
merged.to_excel(r'C:\\Users\\tiago\\cash_circulation\\mergedd.xlsx')